In [ ]:
## Load CERES radiation anomaly dataset
ceres_rad_anom = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ceres_anom/ceres_anom_med_res.nc')

loCld_cre_anom = ceres_rad_anom.loCld_cre
highCld_cre_anom = ceres_rad_anom.highCld_cre
net_cre_anom = ceres_rad_anom.net_cre
clr_sky_anom = ceres_rad_anom.clr_sky
all_sky_anom = ceres_rad_anom.all_sky

In [ ]:
## For each ENSO year, get lags before and after peak SST conditions (lag=0)
en_start_index = []
en_end_index = []
ln_start_index = []
ln_end_index = []

for i in range(0,len(el_nino_yrs)):
    en_start_index.append(int(el_nino_yrs[i][0]))
    en_end_index.append(int(el_nino_yrs[i][1]))
    
    ln_start_index.append(int(la_nina_yrs[i][0]))
    ln_end_index.append(int(la_nina_yrs[i][1]))

In [ ]:
## For each month, composite radiation to get radiation maps as a function of lag
def get_mon_comp(x):
    en_start = x.sel(time=x.time.dt.year.isin(en_start_index))
    en_end = x.sel(time=x.time.dt.year.isin(en_end_index))
    ln_start = x.sel(time=x.time.dt.year.isin(ln_start_index))
    ln_end = x.sel(time=x.time.dt.year.isin(ln_end_index))
    
    start_grp = en_start.groupby('time.month').mean(dim='time') - ln_start.groupby('time.month').mean(dim='time')
    start_grp['month'] = np.arange(-12,0,1)
    start_grp = start_grp.rename({'month':'lag'})

    end_grp = en_end.groupby('time.month').mean(dim='time') - ln_end.groupby('time.month').mean(dim='time')
    end_grp['month'] = np.arange(0,12,1)
    end_grp = end_grp.rename({'month':'lag'})
    
    mon_comp_xr = xr.concat([start_grp,end_grp], dim='lag')
    mon_comp_zonal_xr = mon_comp_xr.mean(dim='longitude') * np.cos(np.deg2rad(new_lat_medRes))
    mon_comp_glb_xr = mon_comp_xr.weighted(weight).mean(dim=('latitude','longitude'))
    
    return mon_comp_xr, mon_comp_zonal_xr, mon_comp_glb_xr

In [ ]:
all_sky_comp, all_sky_comp_zonal, all_sky_comp_glb = get_mon_comp(all_sky_anom)
clr_sky_comp, clr_sky_comp_zonal, clr_sky_comp_glb = get_mon_comp(clr_sky_anom)
net_cre_comp, net_cre_comp_zonal, net_cre_comp_glb = get_mon_comp(net_cre_anom)
loCld_cre_comp, loCld_cre_comp_zonal, loCld_cre_comp_glb = get_mon_comp(loCld_cre_anom)
highCld_cre_comp, highCld_cre_comp_zonal, highCld_cre_comp_glb = get_mon_comp(highCld_cre_anom)

ccf_anoms = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_anom/ccf_anom_med_res.nc')
SST_comp,SST_comp_zonal,SST_comp_glb = get_mon_comp(ccf_anoms.SST_anom)

In [43]:
def get_ceres_cre(lag):
    # Set up subplots figure with PlateCarree projection
    fig, ax = plt.subplots(1, 5, figsize=(24,3), subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})
    fig.tight_layout(w_pad=10)

    vmin = -6
    vmax = 6
        
    lin_title = ['All Sky','Clear Sky','Net CRE','Low Cloud CRE','High Cloud CRE']

    # Plot Trends
    cbar_data = all_sky_comp.sel(lag=lag).plot(ax=ax[0],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                           add_colorbar=False,vmin=vmin,vmax=vmax)
    clr_sky_comp.sel(lag=lag).plot(ax=ax[1],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                           add_colorbar=False,vmin=vmin,vmax=vmax)
    net_cre_comp.sel(lag=lag).plot(ax=ax[2],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                           add_colorbar=False,vmin=vmin,vmax=vmax)
    loCld_cre_comp.sel(lag=lag).plot(ax=ax[3],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                           add_colorbar=False,vmin=vmin,vmax=vmax)
    highCld_cre_comp.sel(lag=lag).plot(ax=ax[4],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                           add_colorbar=False,vmin=vmin,vmax=vmax)

     # Give each plot gridlines, coastlines, and block out continents
    for i in np.arange(0,5,1):
        ax[i].gridlines()
        ax[i].coastlines()
        #ax[i].add_feature(cartopy.feature.LAND,zorder=100,facecolor='k',edgecolor='k')
        ax[i].set_aspect('auto')
        
        if lag == lag_list[0]:
            ax[i].set_title(lin_title[i], fontsize=20)
        elif lag == lag_list[-1]:
            cbar_ax = fig.add_axes([0.01, -0.1, 0.98, 0.05])
            cbar = fig.colorbar(cbar_data, cax=cbar_ax, orientation='horizontal', extend='both')
            cbar.ax.tick_params(labelsize=20)
            cbar.set_label(label='$W/m^2/K$', size=20)
            
            ax[i].set_title(None)
        else:
            ax[i].set_title(None)
    
    fig.suptitle('Lag = ' + str(lag), y=0.7, x=-0.01, rotation='vertical', fontsize=20)
    
    path = '/data/keeling/a/tjhanke2/enso-energy-budget/Output/Plots/monthly_composites/'
#     fig.savefig(path + 'radiation_composites_lag_' + str(lag) + '_mon.png', bbox_inches='tight')

In [ ]:
def get_zonal_ceres_cre():
    # Plot zonal mean vs season
    fig, ax = plt.subplots(1, 5, figsize=(36,9))
    fig.tight_layout(w_pad=9,h_pad=6)
    
    comp_title = ['All-Sky', 'Clear-Sky', 'Net CRE', 'Low Cloud CRE', 'High Cloud CRE']
    vmin = -2
    vmax = 2

    cbar_data = all_sky_comp_zonal.transpose().plot(ax=ax[0],cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)
    
    clr_sky_comp_zonal.transpose().plot(ax=ax[1],cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)
    
    net_cre_comp_zonal.transpose().plot(ax=ax[2],cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)
    
    loCld_cre_comp_zonal.transpose().plot(ax=ax[3],cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)
    
    highCld_cre_comp_zonal.transpose().plot(ax=ax[4],cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)
    
    for i in range(0,5,1):
        ax[i].set_xlabel('Lag (months)', fontsize=17)
        ax[i].set_ylabel('Latitude', fontsize=17)
        ax[i].tick_params(labelsize=15)
        ax[i].set_title('CERES El Nino Zonal Mean ' + comp_title[i] + ' Anomaly vs Season')
        
    cbar_ax = fig.add_axes([1.02, 0, 0.01, 1.01])
    cbar = fig.colorbar(cbar_data, cax=cbar_ax, orientation='vertical', extend='both')
    cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label='$W/m^2/K$', size=20)
    
    path = '/data/keeling/a/tjhanke2/enso-energy-budget/Output/Plots/monthly_composites/'
#     fig.savefig(path + 'zonal_mean_radiation_composites_mon.png', bbox_inches='tight')

In [ ]:
def get_gm_ceres_cre():
    fig, ax = plt.subplots(figsize=(18,12))
    
    ax.plot(lag,all_sky_comp_glb, 'purple', label='ALL SKY')
    ax.plot(lag,clr_sky_comp_glb, 'green', label='CLEAR SKY')
    ax.plot(lag,net_cre_comp_glb, 'black', label='NET CRE')
    ax.plot(lag,loCld_cre_comp_glb, 'red', label='LOW CLOUD CRE')
    ax.plot(lag,highCld_cre_comp_glb, 'blue', label='HIGH CLOUD CRE')
    
    ax.set_xlabel('Lag (months)', fontsize=20)
    ax.set_ylabel('Regression Slope', fontsize=20)
    ax.set_xlim([lag[0],lag[-1]])
    ax.set_ylim([-1.25,0.75])
    ax.axhline(y=0.0, color='k', linestyle='dotted')
    ax.axvline(x=0.0, color='k', linestyle='dotted')
    ax.legend()
        
    ax.set_title('CERES El Nino Global Mean Radiation Anomalies vs Season')
    
    path = '/data/keeling/a/tjhanke2/enso-energy-budget/Output/Plots/monthly_composites/'
#     fig.savefig(path + 'glob_mean_radiation_composites_mon.png', bbox_inches='tight')